In [8]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

**Preprocessing**

In [9]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

In [10]:
train_df = pd.DataFrame(x_train)
train_df['y'] = y_train

test_df = pd.DataFrame(x_test)
test_df['y'] = y_test

In [11]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,y
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72,15.2
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11,42.3
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26,50.0
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01,21.1
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65,17.7


In [12]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,y
0,18.08460,0.0,18.10,0.0,0.679,6.434,100.0,1.8347,24.0,666.0,20.2,27.25,29.05,7.2
1,0.12329,0.0,10.01,0.0,0.547,5.913,92.9,2.3534,6.0,432.0,17.8,394.95,16.21,18.8
2,0.05497,0.0,5.19,0.0,0.515,5.985,45.4,4.8122,5.0,224.0,20.2,396.90,9.74,19.0
3,1.27346,0.0,19.58,1.0,0.605,6.250,92.6,1.7984,5.0,403.0,14.7,338.92,5.50,27.0
4,0.07151,0.0,4.49,0.0,0.449,6.121,56.8,3.7476,3.0,247.0,18.5,395.15,8.44,22.2


In [13]:
scaler = MinMaxScaler(feature_range=(0, 1))

scaled_train = scaler.fit_transform(train_df)
scaled_test = scaler.fit_transform(test_df)

In [17]:
X_train = scaled_train[:,:13]
Y_train = scaled_train[:,-1].reshape(-1,1)
X_test = scaled_test[:,:13]
Y_test = scaled_test[:,-1].reshape(-1,1)

**Building Model**

In [23]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(111, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(43, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])

In [24]:
#defining our own R2 metric for regression evaluation
from tensorflow.keras import backend as K

def det_coeff(y_true, y_pred):
    u = K.sum(K.square(y_true - y_pred))
    v = K.sum(K.square(y_true - K.mean(y_true)))
    return K.ones_like(v) - (u / v)

In [25]:
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, loss='mean_absolute_error', metrics=[det_coeff])

**Training**

In [34]:
model.fit(X_train, Y_train, epochs=50, batch_size=16)

Epoch 1/50
26/26 [==============================] - 0s 1ms/step - loss: 0.0386 - det_coeff: 0.8986
Epoch 2/50
26/26 [==============================] - 0s 3ms/step - loss: 0.0378 - det_coeff: 0.8990
Epoch 3/50
26/26 [==============================] - 0s 2ms/step - loss: 0.0385 - det_coeff: 0.9021
Epoch 4/50
26/26 [==============================] - 0s 3ms/step - loss: 0.0402 - det_coeff: 0.8649
Epoch 5/50
26/26 [==============================] - 0s 3ms/step - loss: 0.0387 - det_coeff: 0.8976
Epoch 6/50
26/26 [==============================] - 0s 2ms/step - loss: 0.0367 - det_coeff: 0.9259
Epoch 7/50
26/26 [==============================] - 0s 1ms/step - loss: 0.0374 - det_coeff: 0.9233
Epoch 8/50
26/26 [==============================] - 0s 2ms/step - loss: 0.0345 - det_coeff: 0.9296
Epoch 9/50
26/26 [==============================] - 0s 3ms/step - loss: 0.0389 - det_coeff: 0.9077
Epoch 10/50
26/26 [==============================] - 0s 3ms/step - loss: 0.0387 - det_coeff: 0.9084
Epoch 11/

**Evaluation**

In [35]:
model.evaluate(X_test, Y_test)

4/4 [==============================] - 0s 1ms/step - loss: 0.0926 - det_coeff: 0.5606


[0.09256098419427872, 0.560624897480011]

**Predictions**

In [36]:
predictions = model.predict(X_test)

In [37]:
y_0 = predictions
y_0 -= scaler.min_[13]
y_0 /= scaler.scale_[13]

y_pred = y_0
y_actual = y_test.reshape(-1,1)

In [38]:
e = y_pred - y_actual

In [39]:
e = e**2

In [40]:
rms = tf.sqrt(e.mean())

In [41]:
error = rms/e.mean()

In [42]:
error.numpy()

0.19040456306857148